In [1]:
 dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

In [113]:
def get_numpy_data(data_frame, features, output):
    if 'constant' not in features:
        features.insert(0, 'constant')
    if 'constant' not in data_frame.columns:
        data_frame['constant'] = 1
    return np.array(data_frame[features]).astype(float), np.array(data_frame[output]).astype(float)

In [11]:
# prediction function
def predict_output(feature_matrix, weights):
    return np.dot(feature_matrix, weights)

In [25]:
import numpy as np
# normalize feature
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    return feature_matrix / norms, norms

In [13]:
# Cost function SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
# The absolute value sign makes the cost function non-differentiable, so simple gradient 
# descent is not viable (you would need to implement a method called subgradient descent)

In [154]:
import pandas as pd
sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)
interested_df, output = get_numpy_data(sales, ['sqft_living', 'bedrooms'], 'price')

In [155]:
normalized_df, norms = normalize_features(interested_df)
initial_weights = [1,4,1]
predictions = predict_output(normalized_df, initial_weights)

In [156]:
# Quiz 1
# ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
r_1 = np.sum((normalized_df[:,1]*(output - predictions + initial_weights[1]*normalized_df[:,1])))
r_2 = np.sum((normalized_df[:,2]*(output - predictions + initial_weights[2]*normalized_df[:,2])))

In [157]:
r_1

87939470.823251754

In [158]:
r_2

80966698.66623947

In [49]:
#       ┌ (ro[i] + lambda/2)    if ro[i] < -lambda/2
#w[i] = ├ 0                     if -lambda/2 <= ro[i] <= lambda/2
#       └ (ro[i] - lambda/2)    if ro[i] > lambda/2

In [52]:
max_lambda = r_1*2
min_lambda = r_2*2

In [53]:
[min_lambda, max_lambda]

[161933397.33247894, 175878941.64650351]

In [159]:
[max(-2*r_1, -2*r_2, 0), max(2*r_1, 2*r_2)]

[0, 175878941.64650351]

In [58]:
# We can say ro[i] quantifies the significance of the i-th feature: the larger ro[i] is the more likely it is for the
# i-th feature to be retained (reserved)

In [59]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    prediction = predict_output(feature_matrix, weights)
    ro_i = np.sum(feature_matrix[:,i]*(output - prediction + weights[i]*feature_matrix[:,i]))
    if i == 0:
        new_weight_i = ro_i
    elif ro_i < -l1_penalty / 2:
        new_weight_i = ro_i + l1_penalty / 2
    elif ro_i > l1_penalty / 2:
        new_weight_i = ro_i - l1_penalty / 2
    else:
        new_weight_i = 0
    
    return new_weight_i

In [62]:
# should print 0.425558846691
import math
print(lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)], [2./math.sqrt(13),3./math.sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.425558846691


In [74]:
# Lasso
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    feature_lenght = len(initial_weights)
    weights = initial_weights
    delta = tolerance + 1
    while delta >= tolerance:
        delta = 0
        for i in range(feature_lenght):
            new_weight_i = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            delta = max(delta, abs(new_weight_i-weights[i]))
            weights[i] = new_weight_i
    return weights

In [116]:
# Model 1
feature_matrix_1, output_1 = get_numpy_data(sales, ['sqft_living', 'bedrooms'], 'price')
normalized_feature_matrix_1, norms_1 = normalize_features(feature_matrix_1)
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0
new_weights = lasso_cyclical_coordinate_descent(normalized_feature_matrix_1, output_1, initial_weights, l1_penalty, tolerance)

In [117]:
np.sum(np.square(predict_output(normalized_feature_matrix_1, new_weights) - output))

1630492476715386.5

In [118]:
new_weights
# bedroom feature's coefficient shrinks to 0

array([ 21624997.95951909,  63157247.20788956,         0.        ])

In [119]:
# Model 2
training = pd.read_csv('kc_house_train_data.csv', dtype=dtype_dict)
test = pd.read_csv('kc_house_test_data.csv', dtype=dtype_dict)

In [120]:
a = 'bedrooms, bathrooms, sqft_living, sqft_lot, floors, waterfront, view, condition, grade, sqft_above, sqft_basement, yr_built, yr_renovated'

In [121]:
features = [x.strip() for x in a.split(',')]

In [122]:
training_feature_matrix, training_output = get_numpy_data(training, features, 'price')

In [123]:
normalized_feature_matrix, norms = normalize_features(training_feature_matrix)

In [127]:
initial_weights = np.zeros(len(features))
l1_penalty = 1e7
tolerance=1
weights1e7 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, training_output, initial_weights, l1_penalty, tolerance)

In [129]:
np.where(weights1e7 != 0)

(array([0, 3, 6, 7], dtype=int64),)

In [130]:
features

['constant',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated']

In [148]:
initial_weights = np.zeros(len(features))
l1_penalty = 1e8
tolerance=1
weights1e8 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, training_output, initial_weights, l1_penalty, tolerance)

In [149]:
np.where(weights1e8 != 0)

(array([0], dtype=int64),)

In [137]:
initial_weights = np.zeros(len(features))
l1_penalty = 1e4
tolerance= 5e5
weights1e4 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, training_output, initial_weights, l1_penalty, tolerance)

In [150]:
np.where(weights1e4 != 0)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13], dtype=int64),)

In [140]:
normalized_weights1e7 = weights1e7 / norms
normalized_weights1e8 = weights1e8 / norms
normalized_weights1e4 = weights1e4 / norms

In [141]:
# Compare models on TEST data
test_feature_matrix, test_output = get_numpy_data(test, features, 'price')

In [144]:
rss_7 = np.sum(np.square(predict_output(test_feature_matrix, normalized_weights1e7) - test_output))
rss_8 = np.sum(np.square(predict_output(test_feature_matrix, normalized_weights1e8) - test_output))
rss_4 = np.sum(np.square(predict_output(test_feature_matrix, normalized_weights1e4) - test_output))

In [145]:
rss_7

275962075920366.78

In [146]:
rss_8

537166151497322.75

In [147]:
rss_4

228459958971393.25